In [67]:
from dataclasses import dataclass
from typing import Callable
from random import randint
from random import sample
from random import random
from pprint import pprint
from copy import deepcopy

In [68]:
from dataclasses import dataclass
from typing import Callable


@dataclass
class Cromossomo:
    dados: list[int]
    fitness: float = 0


@dataclass
class Config:
    tam_cromossomo: int
    tam_populacao: int
    max_iteracoes: int
    max_congelamento : int
    fitness: Callable[[Cromossomo], float]

    selecionar_pais: Callable[[list[Cromossomo]], list[tuple[Cromossomo, Cromossomo]]]

    aplicar_cruzamento: Callable[[list[tuple[Cromossomo, Cromossomo]]], list[Cromossomo]]

    aplicar_mutacao: Callable[[list[Cromossomo], float], None]
    taxa_mutacao: float

    selecionar_sobreviventes: Callable[[list[Cromossomo]], list[Cromossomo]]

    def inicializar_populacao(tam_populacao, tam_cromossomo) -> list[Cromossomo]:
        P0 = []
        for i in range(tam_populacao):
            cromossomo = Cromossomo([randint(0, 1) for j in range(tam_cromossomo)])
            P0.append(cromossomo)
        return P0

In [69]:
def inicializar_populacao(tam_populacao, tam_cromossomo) -> list[Cromossomo]:
    P0 = []
    for i in range(tam_populacao):
        cromossomo = Cromossomo([randint(0, 1) for j in range(tam_cromossomo)])
        P0.append(cromossomo)
    return P0

In [70]:
def torneio(populacao: list[Cromossomo], tam_torneio: int = 3) -> list[tuple[Cromossomo, Cromossomo]]:
    casais = []

    for i in range(len(populacao) // 2):

        torneio1 = sample(populacao, tam_torneio * 2)
        pai1 = sorted(torneio1, key=lambda x: x.fitness, reverse=True)[0] 
        
        torneio2 = sample(populacao, tam_torneio)
        pai2 = sorted(torneio2, key=lambda x: x.fitness, reverse=True)[0] 

        casais.append((pai1, pai2))
    
    return casais

In [71]:
def crossover_1_corte(casais):
    filhos = []

    for par in casais:

        # Desempacota os pais
        pai1 = par[0].dados
        pai2 = par[1].dados

        # Sorteia o ponto de corte
        corte = randint(1, len(pai1) - 1)

        filho1 = pai1[:corte] + pai2[corte:] 
        filho2 = pai2[:corte] + pai1[corte:] 

        filhos.append(Cromossomo(filho1))
        filhos.append(Cromossomo(filho2))

    return filhos

In [72]:
def mutacao(cromossomos: list[Cromossomo], taxa_mutacao: float):
    for cromossomo in cromossomos:
        for i, c in enumerate(cromossomo.dados):
            if random() < taxa_mutacao:
                if c == 1:
                    cromossomo.dados[i] = 0
                else:
                    cromossomo.dados[i] = 1

In [73]:
def elitismo(populacao):
    populacao.sort(key=lambda x: x.fitness, reverse=True)
    return populacao[:len(populacao)//2]

In [74]:
def maximizar_1s(cromossomo: Cromossomo) -> int:
    """
    Função de avaliação do problema. É nessa função que o cromossomo é interpretado
    para calcular a aptidão e verificar se está atingindo o objetivo.
    """
    return sum(cromossomo.dados)

In [75]:
def fitness_cartoes(cromossomo):
    pilha1 = []
    pilha2 = []

    for i, cartao in enumerate(cromossomo.dados):
        if cartao == 0:
            pilha1.append(i + 1)  # Adicione 1 para representar os números de cartões de 1 a 10
        else:
            pilha2.append(i + 1)

    soma_primeira_pilha = sum(pilha1)
    produto_segunda_pilha = 1 if not pilha2 else 1  # Evite a multiplicação por 0
    for cartao in pilha2:
        produto_segunda_pilha *= cartao

    fitness_soma = abs(36 - soma_primeira_pilha)
    fitness_produto = abs(360 - produto_segunda_pilha)

    # Queremos minimizar ambas as diferenças, então podemos somar os inversos das fitness.
    return 1 / (1 + fitness_soma + fitness_produto)


In [76]:
def algoritmo_genetico(config: Config) -> list[int]:
    """
    Implementação do algoritmo genético clássico.

    Arguments:
        config: Parâmetros de configuração do AG.
    """

    # 1. t = 0
    t = 0
    
    # 2. Inicializar a população inicial P_0
    P = inicializar_populacao(config.tam_populacao, config.tam_cromossomo)

    # a. Avaliar a população(Pt)
    for c in P:
        c.fitness = config.fitness(c)

    # Salva o melhor indivíduo 
    P.sort(key=lambda x: x.fitness, reverse=True)

    melhor_individuo = deepcopy(P[0])
    cont_congelamento = 0
    
    
    # 3. Enquanto critério de parada == falso
    terminou = False
    while (t < config.max_iteracoes 
               and cont_congelamento < config.max_congelamento):
        
        # b. P' = Selecionar pais(Pt)
        casais = config.selecionar_pais(P)
            
        # c. F = Aplicar recombinação e mutação(P')
        F = config.aplicar_cruzamento(casais)
        config.aplicar_mutacao(F, config.taxa_mutacao)
        
        # d. Avaliar a população(F)
        for c in F:
            c.fitness = config.fitness(c)
        
        # e. Pt+1 = Selecionar sobreviventes(Pt + F)
        P = config.selecionar_sobreviventes(P + F)

        # Salva o melhor indivíduo 
        P.sort(key=lambda x: x.fitness, reverse=True)

        if P[0].fitness > melhor_individuo.fitness:
            melhor_individuo = deepcopy(P[0])
            cont_congelamento = 0
        else:
            cont_congelamento += 1

        print(f'{t:04d} - {melhor_individuo.fitness} - {melhor_individuo.dados}')
        
        # f. t = t + 1
        t = t + 1
        
    return melhor_individuo

In [77]:
config = Config(
    tam_cromossomo=10,
    tam_populacao=100,
    
    max_iteracoes=1000,
    max_congelamento=100,
    
    fitness=fitness_cartoes,

    taxa_mutacao=0.05,
    
    selecionar_pais=torneio,

    aplicar_cruzamento=crossover_1_corte,
    aplicar_mutacao=mutacao,
    selecionar_sobreviventes=elitismo,
)

In [78]:
# Execução do algoritmo genético
solucao = algoritmo_genetico(config)
print('Solução encontrada: ')
solucao

0000 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0001 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0002 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0003 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0004 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0005 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0006 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0007 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0008 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0009 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0010 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0011 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0012 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0013 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0014 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0015 - 0.0025575447570332483 - [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0016 - 0

Cromossomo(dados=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], fitness=0.0025575447570332483)